In [1]:
import numpy as np 
import pandas as pd
import calendar
import math

# plots
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

In [2]:
filenames = ['properties/properati-AR-2016-03-01-properties-sell-six_months.csv',
            'properties/properati-AR-2016-04-01-properties-sell.csv',
            'properties/properati-AR-2016-05-01-properties-sell.csv',
            'properties/properati-AR-2016-06-01-properties-sell.csv',
            'properties/properati-AR-2016-07-01-properties-sell.csv',
            'properties/properati-AR-2016-08-01-properties-sell.csv',
            'properties/properati-AR-2016-09-01-properties-sell.csv',
            'properties/properati-AR-2016-10-01-properties-sell.csv',
            'properties/properati-AR-2016-11-01-properties-sell.csv',
            'properties/properati-AR-2016-12-01-properties-sell.csv', 
            'properties/properati-AR-2017-01-01-properties-sell.csv',
            'properties/properati-AR-2017-02-01-properties-sell.csv',
            'properties/properati-AR-2017-08-01-properties-sell-six_months.csv']

dataframes = []
for filename in filenames:
    dataframes.append(pd.read_csv(filename))
    
#Uno los dataFrame
metadata = pd.DataFrame()
for dataframe in dataframes:
    metadata = pd.concat([metadata, dataframe]).drop_duplicates().reset_index(drop=True)

In [3]:
data = metadata.loc[(metadata.state_name.str.contains('Buenos Aires')) | (metadata.state_name.str.contains\
                ('Capital Federal'))  | (metadata.state_name.str.contains\
                ('Bs.As') ),:]
data.shape
# 1.283.741 entries antes, ahora 242290

(242290, 27)

In [4]:
test = pd.read_csv('properties/properati_dataset_testing_noprice.csv')
test_visualiz = test.loc[:,['state_name', 'id']].groupby('state_name').agg(np.size)
test_visualiz.head(10)

,id
state_name,
Bs.As. G.B.A. Zona Norte,2
Bs.As. G.B.A. Zona Oeste,3101
Bs.As. G.B.A. Zona Sur,2215
Buenos Aires Interior,1
Capital Federal,8847


In [5]:
data['state_name'].tail()

296472                 Capital Federal
296473        Bs.As. G.B.A. Zona Norte
296474                 Capital Federal
296475    Buenos Aires Costa Atlántica
296476                 Capital Federal
Name: state_name, dtype: object

In [7]:
# Depuración del set de test
#Completo los valores NaN con promedios para realizar la predicción
prom_surface_total_in_m2 = test['surface_total_in_m2'].mean()
prom_surface_covered_in_m2 = test['surface_covered_in_m2'].mean()
prom_lat = test['lat'].mean()
prom_lon = test['lon'].mean()

test_float = test
test_float['surface_total_in_m2'].fillna(prom_surface_total_in_m2, inplace=True)
test_float['surface_covered_in_m2'].fillna(prom_surface_covered_in_m2, inplace=True)
test_float['lat'].fillna(prom_lat, inplace=True)
test_float['lon'].fillna(prom_lon, inplace=True)

test_float['fecha'] = pd.to_datetime(test_float['created_on'])
test_float['anio'] = test_float['fecha'].map(lambda x:x.year)
test_float['mes'] = test_float['fecha'].map(lambda x:x.month)

test_float.head(100)

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,fecha,anio,mes
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.000000,359.471588,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...,2017-08-24,2017,8
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,-34.629923,-58.465820,0.000000,359.471588,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...,2017-08-25,2017,8
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.000000,48.000000,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...,2017-08-01,2017,8
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,51.000000,51.000000,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...,2017-08-01,2017,8
5,2263407,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
6,2263408,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
7,2263409,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...,2017-08-01,2017,8
8,2263410,2017-08-01,departamento,venta,Colegiales,|Argentina|Capital Federal|Colegiales|,Argentina,Capital Federal,"-34.5787213694,-58.4570952614",-34.578721,-58.457095,67.000000,67.000000,NaN,NaN,2600,EXCELENTE 2 AMB- APTO PROFESIO...,2017-08-01,2017,8
9,2263411,2017-08-01,departamento,venta,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,47.000000,47.000000,NaN,NaN,NaN,IMPECABLE MONOAMBIENTE- DIVISI...,2017-08-01,2017,8


In [7]:
test_float.count()

id                         14166
created_on                 14166
property_type              14166
operation                  14166
place_name                 14166
place_with_parent_names    14166
country_name               14166
state_name                 14166
lat-lon                    10487
lat                        14166
lon                        14166
surface_total_in_m2        14166
surface_covered_in_m2      14166
floor                       1368
rooms                       7500
expenses                    2543
description                14166
dtype: int64

<br><B><FONT FACE="Arial" SIZE="5">Preparo los datos</FONT></B><br/>

In [9]:
data_float = data.loc[:, ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',\
                          'lat', 'lon', 'created_on']].dropna()

# filtro de datos anómalos
data_float = \
data_float.loc[((data_float.surface_total_in_m2 >= 0.0) & (data_float.surface_total_in_m2 <= 1000000.0)) & \
               ((data_float.surface_covered_in_m2 >= 0.0) & (data_float.surface_covered_in_m2 <= 50000000.0)) \
               , :]


data_float.shape
#(104604, 6)

(104604, 6)

In [10]:
data_float['fecha'] = pd.to_datetime(data_float['created_on'])
data_float['anio'] = data_float['fecha'].map(lambda x:x.year)
data_float['mes'] = data_float['fecha'].map(lambda x:x.month)


## Uso PCA como reductor de dimensiones para sumar dos nuevas dimensiones al dataset.
#### Para ello utilizamos las features 'surface_total_in_m2', 'surface_covered_in_m2', 'lat' y 'lon'

In [11]:
data_float_pca = data_float.loc[:,['surface_total_in_m2', 'surface_covered_in_m2',\
                          'lat', 'lon']]
data_float_pca

,surface_total_in_m2,surface_covered_in_m2,lat,lon
0,500.0,500.0,-34.643701,-58.651548
5,280.0,100.0,-34.536689,-58.566976
9,45.0,42.0,-34.603723,-58.381593
13,135.0,135.0,-34.546433,-58.545713
14,330.0,310.0,-34.345731,-58.855461
15,0.0,218.0,-34.735512,-58.398874
16,30.0,30.0,-34.649843,-58.657190
17,47.0,47.0,-34.651906,-58.644602
18,0.0,136.0,-34.797334,-58.395871
19,294.0,100.0,-34.767167,-58.215465


In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(data_float_pca)
red = pca.transform(data_float_pca)
print(pca.singular_values_)

[ 284133.05197835   90861.62369182]


In [13]:
red

array([[ 361.68908677,  340.42935535],
       [  91.23008934,  -27.33677514],
       [-149.33948855,  -54.08148562],
       ..., 
       [-100.72495834,  -37.29972439],
       [-150.7235221 ,  -56.9248006 ],
       [-148.74072438,  -57.18655313]])

In [14]:
d = {'col1': red[:,0], 'col2': red[:,1]}
red_data = pd.DataFrame(data=d)

data_float.reset_index(inplace=True)
data_float['PCA1'] = red_data['col1']
data_float['PCA2'] = red_data['col2']
data_float.tail()

,index,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,lat,lon,created_on,fecha,anio,mes,PCA1,PCA2
104599,296382,210000.0,403.0,403.0,-34.637014,-58.433177,2017-08-01,2017-08-01,2017,8,252.828236,256.958900
104600,296388,325000.0,113.0,113.0,-34.590628,-58.397539,2017-08-01,2017-08-01,2017,8,-72.632032,7.408077
104601,296410,165000.0,91.0,65.0,-34.599214,-58.402519,2017-08-01,2017-08-01,2017,8,-100.724958,-37.299724
104602,296415,165000.0,44.0,39.0,-34.587425,-58.397372,2017-08-01,2017-08-01,2017,8,-150.723522,-56.924801
104603,296474,131500.0,46.0,39.0,-34.570639,-58.475596,2017-08-01,2017-08-01,2017,8,-148.740724,-57.186553


In [15]:
test_float.count()

id                         14166
created_on                 14166
property_type              14166
operation                  14166
place_name                 14166
place_with_parent_names    14166
country_name               14166
state_name                 14166
lat-lon                    10487
lat                        14166
lon                        14166
surface_total_in_m2        14166
surface_covered_in_m2      14166
floor                       1368
rooms                       7500
expenses                    2543
description                14166
fecha                      14166
anio                       14166
mes                        14166
dtype: int64

In [16]:
test_float_pca = test_float.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                    'lat', 'lon']]
test_float_pca

,surface_total_in_m2,surface_covered_in_m2,lat,lon
0,0.0,359.471588,-34.610988,-58.363464
1,0.0,359.471588,-34.629923,-58.465820
2,53.0,48.000000,-34.589363,-58.412880
3,39.0,39.000000,-34.629923,-58.465820
4,51.0,51.000000,-34.629923,-58.465820
5,39.0,39.000000,-34.629923,-58.465820
6,39.0,39.000000,-34.629923,-58.465820
7,39.0,39.000000,-34.629923,-58.465820
8,67.0,67.000000,-34.578721,-58.457095
9,47.0,47.000000,-34.629923,-58.465820


In [17]:
red_test = pca.transform(test_float_pca)
red_test

array([[-152.40226211,  266.54887287],
       [-152.40226019,  266.5488747 ],
       [-140.6230308 ,  -49.18011945],
       ..., 
       [  35.33552885,   15.34572404],
       [-131.70044467,  -50.35802059],
       [ 181.40810645,   -8.98125454]])

In [18]:
d = {'col1': red_test[:,0], 'col2': red_test[:,1]}
red_test_data = pd.DataFrame(data=d)

#test_float.reset_index(inplace=True)
test_float['PCA1'] = red_test_data['col1']
test_float['PCA2'] = red_test_data['col2']
test_float.tail()

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,...,surface_covered_in_m2,floor,rooms,expenses,description,fecha,anio,mes,PCA1,PCA2
14161,2245938,2017-08-01,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6115752,-58.3630313",-34.611575,...,96.0,NaN,NaN,8500,"Edificio.""Terrazas del Dique"".Al Frente.Balcón...",2017-08-01,2017,8,-78.822558,-8.922186
14162,2245978,2017-08-17,ph,venta,Temperley,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,"-34.7821307,-58.4157403",-34.782131,...,26.0,NaN,2.0,NaN,Venta de imperdible PH con patio 2 AMBIENTE...,2017-08-17,2017,8,-158.373327,-69.027727
14163,2246083,2017-08-01,casa,venta,Lomas de Zamora,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,"-34.7585252,-58.4097202",-34.758525,...,135.0,NaN,NaN,NaN,"Venta de Casa 4 AMBIENTES en Lomas de Zamora, ...",2017-08-01,2017,8,35.335529,15.345724
14164,2246089,2017-08-25,departamento,venta,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,"-34.563221078,-58.4610526805",-34.563221,...,48.0,NaN,NaN,4128,DEPARTAMENTO EN VENTA / ALQUILER | BELGRANO ...,2017-08-25,2017,8,-131.700445,-50.358021
14165,2246127,2017-08-12,casa,venta,Ciudad Evita,|Argentina|Bs.As. G.B.A. Zona Oeste|La Matanza...,Argentina,Bs.As. G.B.A. Zona Oeste,"-34.7091916902,-58.5302893817",-34.709192,...,130.0,NaN,4.0,NaN,Venta de Chalet 4 AMBIENTES en CIRCUNSCRIPCIÓN...,2017-08-12,2017,8,181.408106,-8.981255


<br><B><FONT FACE="Arial" SIZE="5">Predicción usando XGB</FONT></B><br/>

In [25]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'PCA1', 'PCA2']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [19]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls', 'criterion': 'mse'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [20]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.838/0.717
#Precisión Gradient Boosting train/test  0.686/0.325 cuando la precisión era mejor, nuestras predicciones eran 
#peores. Habría que ver cómo mide la precisión ésta cosa

Precisión Gradient Boosting train/test  0.720/0.324


In [21]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'rooms', 'PCA1', 'PCA2']])
test['price_aprox_usd_XGB'].head(100)

0     462710.462828
1     462710.462828
2      93127.951932
3      78135.736865
4     110383.583554
5      78135.736865
6      78135.736865
7      78135.736865
8     116384.416076
9      93127.951932
10     93127.951932
11     93127.951932
12    116384.416076
13    227772.812422
14     78135.736865
15     78135.736865
16     78135.736865
17     78135.736865
18     78135.736865
19     78135.736865
20     78135.736865
21     78135.736865
22    110383.583554
23    110383.583554
24    110383.583554
25    110383.583554
26    110383.583554
27     78135.736865
28     78135.736865
29     78135.736865
          ...      
70    116384.416076
71     96432.766677
72    167214.810031
73    167214.810031
74    116384.416076
75    116384.416076
76    116384.416076
77    116384.416076
78    184809.902605
79    116384.416076
80    156401.878157
81    184809.902605
82    199337.873131
83    153016.308900
84    153016.308900
85     93127.951932
86    183537.676882
87     78135.736865
88    158677.125962


In [22]:
#Genero el csv con los nombres de columnas requeridos por Kaggle
#test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
#test_float_reduced.columns=['id', 'price_usd']
#test_float_reduced.to_csv('SubmitXGBoostingBuenosAires.csv', index=False)

<br><B><FONT FACE="Arial" SIZE="5">XGBoosting con lat y lon</FONT></B><br/>

In [36]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'lat', 'lon', 'PCA1', 'PCA2', 'mes']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [37]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [38]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.905/0.736
#Precisión Gradient Boosting train/test  0.864/0.549


#Precisión Gradient Boosting train/test  1.000/0.963

Precisión Gradient Boosting train/test  0.840/0.508


In [39]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                         'lat', 'lon', 'PCA1', 'PCA2', 'mes']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
#test_float_reduced.to_csv('SubmitXGBoostingLatLonGenerico.csv', index=False)
test_float_reduced['price_usd'].head(100)

0     688283.125618
1     387628.013086
2     148279.600078
3      89033.151932
4     114116.045895
5      89033.151932
6      89033.151932
7      89033.151932
8     198299.997250
9     105098.728570
10    105098.728570
11    105098.728570
12    122834.298047
13    530656.385116
14     89033.151932
15     89033.151932
16     89033.151932
17     89033.151932
18     89033.151932
19     89033.151932
20     89033.151932
21     89033.151932
22    112486.519563
23    114116.045895
24    114116.045895
25    114116.045895
26    114116.045895
27     89033.151932
28     89033.151932
29    129075.705791
          ...      
70    121383.128667
71    158195.307064
72    137549.438211
73    137549.438211
74    121383.128667
75    121383.128667
76    121383.128667
77    121383.128667
78    315216.561990
79    121383.128667
80    133386.738780
81    315216.561990
82    171529.478707
83    119379.362139
84    237832.031394
85    104076.503847
86    266557.715748
87    129075.705791
88    132896.351979


<br><B><FONT FACE="Arial" SIZE="5">Uso grid-search para estimar los parametros</FONT></B><br/>

In [40]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2',  'lat', 'lon', 'PCA1', 'PCA2', 'mes']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd



In [41]:
#Grid-search para encontrar el parametro n_estimators
#Arranco con parametros normales al azar.
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

param_test1 = {'n_estimators':list(range(1,600,10))}

gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=500,\
min_samples_leaf=50, max_depth=8, max_features='sqrt', subsample=0.8, random_state=10),\
param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)

#gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1 ,max_features='sqrt',\
#subsample=0.8, random_state=10), param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
                        
gsearch1.fit(X_train, y_train)
#Me fijo el resultado del parametro

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=50, min_samples_split=500,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=10, subsample=0.8, verbose=0,
             warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 201, 211, 221, 231, 241, 251, 261, 271, 281, 291, 301, 311, 321, 331, 341, 351, 361, 371, 381, 391, 401, 411, 421, 431, 441, 451, 461, 471, 481, 491, 501, 511, 521, 531, 541, 551, 561, 571, 581, 591]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=0)

In [42]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_


([mean: -69311770664.06862, std: 38246419668.60260, params: {'n_estimators': 1},
  mean: -52807260844.38239, std: 35415660111.73663, params: {'n_estimators': 11},
  mean: -47177009966.61791, std: 34019245766.81974, params: {'n_estimators': 21},
  mean: -44396630336.40665, std: 32466634790.07997, params: {'n_estimators': 31},
  mean: -42759214254.60387, std: 30657875539.02027, params: {'n_estimators': 41},
  mean: -42433972949.62803, std: 30565975468.79169, params: {'n_estimators': 51},
  mean: -42254972442.33250, std: 30069400392.89237, params: {'n_estimators': 61},
  mean: -42114602485.66200, std: 30031855480.53352, params: {'n_estimators': 71},
  mean: -41804881763.50635, std: 29960364810.03455, params: {'n_estimators': 81},
  mean: -41918029896.09380, std: 29890229472.89849, params: {'n_estimators': 91},
  mean: -41609270207.48068, std: 29452243396.01761, params: {'n_estimators': 101},
  mean: -41550753087.02299, std: 29207487112.02265, params: {'n_estimators': 111},
  mean: -412621

In [43]:
#Encontrar los parametros max_depth y min_samples_split
#Aca ya uso n_estimators=491
param_test2 = {'max_depth':list(range(2,16,2)), 'min_samples_split':list(range(200,1001,10))}
gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=431, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: -44053317323.73799, std: 27456000787.58194, params: {'min_samples_split': 200, 'max_depth': 2},
  mean: -43564379869.28528, std: 27568081053.17211, params: {'min_samples_split': 210, 'max_depth': 2},
  mean: -43437344695.50009, std: 27597679628.44562, params: {'min_samples_split': 220, 'max_depth': 2},
  mean: -43828366860.70754, std: 27306063915.06524, params: {'min_samples_split': 230, 'max_depth': 2},
  mean: -43813989395.39293, std: 27530625966.19358, params: {'min_samples_split': 240, 'max_depth': 2},
  mean: -44326720414.16336, std: 27450871967.54739, params: {'min_samples_split': 250, 'max_depth': 2},
  mean: -44200794646.17140, std: 28024179370.22036, params: {'min_samples_split': 260, 'max_depth': 2},
  mean: -44655483756.73922, std: 27553345539.47282, params: {'min_samples_split': 270, 'max_depth': 2},
  mean: -44645607159.50834, std: 27625647826.77510, params: {'min_samples_split': 280, 'max_depth': 2},
  mean: -44864625850.00218, std: 27638161508.71778, params: {'mi

In [44]:
#Encontrar el parametro min_samples_leaf
# 'n_estimators': 491, 'max_depth': 12, 'min_samples_split': 210

param_test3 = {'min_samples_leaf':list(range(10,100,1))}
gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=431, min_samples_split=240, max_depth=4,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train, y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: -40604892016.38500, std: 23549860420.33123, params: {'min_samples_leaf': 10},
  mean: -40889929270.81602, std: 23814681974.98995, params: {'min_samples_leaf': 11},
  mean: -41215661869.61650, std: 23327385115.44194, params: {'min_samples_leaf': 12},
  mean: -41189374586.51842, std: 23978064837.42236, params: {'min_samples_leaf': 13},
  mean: -40985077239.36782, std: 23981914040.59245, params: {'min_samples_leaf': 14},
  mean: -41641931613.39547, std: 24206549872.72424, params: {'min_samples_leaf': 15},
  mean: -41371555752.37747, std: 24605825967.80236, params: {'min_samples_leaf': 16},
  mean: -40989168931.89845, std: 24665855737.39567, params: {'min_samples_leaf': 17},
  mean: -40929042650.97029, std: 24579498504.39795, params: {'min_samples_leaf': 18},
  mean: -40441878047.79842, std: 24773324843.32962, params: {'min_samples_leaf': 19},
  mean: -40869123738.34986, std: 25293355593.01587, params: {'min_samples_leaf': 20},
  mean: -40398805606.97884, std: 24542320903.94664, pa

In [46]:
#Encontrar el parametro max_features
# 'n_estimators': 491, 'max_depth': 12, 'min_samples_split': 210, 'min_samples_leaf': 65

param_test4 = {'max_features':list(range(1,6,1))}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=431, max_depth=4, min_samples_split=240, min_samples_leaf=48
                                                              , subsample=0.8, random_state=10),
param_grid = param_test4, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train, y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: -40609018639.33858, std: 26018628614.34220, params: {'max_features': 1},
  mean: -38937005164.30394, std: 25022471231.09821, params: {'max_features': 2},
  mean: -39814265971.18559, std: 25597289427.90397, params: {'max_features': 3},
  mean: -39399491597.86182, std: 23611852957.18428, params: {'max_features': 4},
  mean: -39765966069.20859, std: 24513952620.01303, params: {'max_features': 5}],
 {'max_features': 2},
 -38937005164.30394)

In [47]:
# 'n_estimators': 491, 'max_depth': 12, 'min_samples_split': 210, 'min_samples_leaf': 65, 'max_features': 4
#Voy a realizar una prueba con estos parametros
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 431, 'max_depth': 4, 'min_samples_split': 200, 'min_samples_leaf':42 , 'max_features': 2,
          'learning_rate': 0.1}


clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

#Me fijo la precision
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.823/0.767
#Precisión Gradient Boosting train/test  0.875/0.526

Precisión Gradient Boosting train/test  0.771/0.485


In [48]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                         'lat', 'lon', 'PCA1', 'PCA2', 'mes']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoostingGS.csv', index=False)

In [50]:
# Imprimo la relevancia de cada feature para el regresor
# feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2',  'lat', 'lon', 'PCA1', 'PCA2', 'mes']

print clf.feature_importances_

[ 0.15799698  0.17689591  0.14192077  0.11627962  0.15937862  0.23092176
  0.01660636]


## Uso grid-search para estimar los parametros, ésta vez todos juntos


In [35]:
#Creo X e Y
#feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms',  'lat', 'lon']
#TRAIN = data_float[:2600]
#TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
#X_train = TRAIN[feature_cols]
#X_test = TEST[feature_cols]

#y_train = TRAIN.price_aprox_usd
#y_test = TEST.price_aprox_usd

In [36]:
#Grid-search para encontrar el parametro n_estimators
#Arranco con parametros normales al azar.
#from sklearn.grid_search import GridSearchCV
#from sklearn.ensemble import GradientBoostingRegressor

#param_test1 = {'n_estimators':list(range(1,500,5)), 'max_depth':list(range(2,16,2)),\
#               'min_samples_split':list(range(200,1000,50)), 'min_samples_leaf':list(range(10,100,5)),\
#                'max_features':list(range(1,4,1))}

#gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, subsample=0.8, random_state=10),\
#                param_grid = param_test1, scoring='neg_mean_squared_error', n_jobs=4,iid=False, cv=5)
                        
#gsearch1.fit(X_train, y_train)

IndentationError: unexpected indent (<ipython-input-36-5066f7779619>, line 11)

In [ ]:
# {'max_depth': 10,
#  'max_features': 3,
#  'min_samples_leaf': 40,
#  'min_samples_split': 200,
#  'n_estimators': 261},

#gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

### Con estos hiper-parametros estimados, hago una prediccion

In [ ]:
# 'n_estimators':261, 'max_depth': 10, 'min_samples_split': 200, 'min_samples_leaf': 40, 'max_features': 3
#Voy a realizar una prueba con estos parametros
#Entreno el modelo
#from sklearn import ensemble
#params = {'n_estimators': 261, 'max_depth': 10, 'min_samples_split': 200, 'min_samples_leaf': 40, 'max_features': 3,
#          'learning_rate': 0.1}


#clf = ensemble.GradientBoostingRegressor(**params)

#clf.fit(X_train, y_train)

#Me fijo la precision
#print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
#      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.857/0.837

In [ ]:
#Hago la prediccion para Kaggle
#test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
#                                        'floor', 'rooms',  'lat', 'lon']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
#test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
#test_float_reduced.columns=['id', 'price_usd']
#test_float_reduced.to_csv('SubmitXGBoostingGS2.csv', index=False)